In [1]:
#! pip install py2neo
from py2neo import Graph
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import numpy as np


  Using cached https://files.pythonhosted.org/packages/fa/37/45185cb5abbc30d7257104c434fe0b07e5a195a6847506c074527aa599ec/Click-7.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/87/61/2dfea88583d5454e3a64f9308a686071d58d59a55db638268a6413e1eb6d/prompt_toolkit-2.0.10-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/13/e5/6d710c9cf96c31ac82657bcfb441df328b22df8564d58d0c4cd62612674c/Pygments-2.3.1-py2.py3-none-any.whl
jupyter-console 5.2.0 has requirement prompt_toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 2.0.10 which is incompatible.
ipython 6.4.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 2.0.10 which is incompatible.
  Found existing installation: click 6.7
    Uninstalling click-6.7:
      Successfully uninstalled click-6.7
  Found existing installation: prompt-toolkit 1.0.15
    Uninstalling prompt-toolkit-1.0.15:
      Successfully uninstalled prompt-toolkit-1.0.15
  Found existing inst

In [28]:
graph = Graph("bolt://3.220.233.169:7687", auth=("neo4j", "i-0e23d19f0d8795714"))

In [29]:
graph.run("CALL db.schema()").data()


[{'nodes': [(_-22:Category {constraints: [], indexes: ['name'], name: 'Category'}),
   (_-23:User {constraints: ['CONSTRAINT ON ( user:User ) ASSERT user.id IS UNIQUE'], indexes: [], name: 'User'}),
   (_-24:Review {constraints: ['CONSTRAINT ON ( review:Review ) ASSERT review.id IS UNIQUE'], indexes: [], name: 'Review'}),
   (_-25:Reputation {constraints: [], indexes: [], name: 'Reputation'}),
   (_-21:Business {constraints: ['CONSTRAINT ON ( business:Business ) ASSERT business.id IS UNIQUE'], indexes: [], name: 'Business'})],
  'relationships': [(Review)-[:REVIEWS {}]->(Business),
   (User)-[:WROTE {}]->(Review),
   (User)-[:FRIEND {}]->(User),
   (User)-[:COOL_LEVEL {}]->(Reputation),
   (User)-[:FUNNY_LEVEL {}]->(Reputation),
   (Business)-[:IN_CATEGORY {}]->(Category)]}]

now trying to automate atleast some of the process for the other attributes

| qunatile | ident | u.cool | u.funny | u.useful |
| --- | --- | --- | --- | --- |
| 0.20 | 1072521.40	| 0.0 | 0.0 | 0.0 |
| 0.35 | 1968091.95 | 0.0	 |0.0	|1.0 |
| 0.50 |	3063662.50	| 0.0	 | 0.0	| 3.0 | 
| 0.65 |	4389233.05	| 1.0	 | 1.0	| 7.0 |
| 0.80 |	5984803.60	| 5.0	 | 4.0	| 19.0 |
| 0.95 |	7830374.15	| 36.0	| 33.0	| 110.0 |


In [24]:
#the 0 group needs be done seperately because of the way cypher handles nodes that do not have a funny value.
# graph.run("""
# MATCH (n)
# WHERE ID(n)= 6543
# MATCH (u:User) WHERE NOT exists(u.funny)
# CREATE (u)-[:FUNNY_LEVEL]->(n)
# """)

In [ ]:
#this is for testing the funny edge adding loop
id_list=[(6543,0,1),(6668,1,2),(6552,2,33),(6561,33,999999)]
dflist=[]
for index in id_list:
    tempdf=graph.run(f"\
    MATCH (n)\
    WHERE ID(n)={index[0]}\
    MATCH (u:User) WHERE {index[1]}=<u.funny<{index[2]}\
    RETURN n.funny, u.name, u.funny LIMIT 6").to_data_frame()
    dflist.append(tempdf)

In [25]:
#this is for adding the funny edges
# id_list=[(6668,1,2),(6552,2,33),(6561,33,999999)]
# dflist=[]



# for index in id_list:
#     tempdf=graph.run(f"\
#     MATCH (n)\
#     WHERE ID(n)={index[0]}\
#     MATCH (u:User) WHERE {index[1]}<=u.funny<{index[2]}\
#     CREATE (u)-[:FUNNY_LEVEL]->(n)").to_data_frame()
#     dflist.append(tempdf)


In [34]:
#the 0 group needs be done seperately because of the way cypher handles nodes that do not have a useful value.
# graph.run("""
# MATCH (n)
# WHERE ID(n)= 6670
# MATCH (u:User) WHERE NOT exists(u.useful)
# CREATE (u)-[:USEFUL_LEVEL]->(n)
# """)

In [32]:
#this is for testing the funny edge adding loop
id_list=[(6674,0,1),(6680,1,3),(6681,3,7),(6682,7,19),(6683,110,999999)]
dflist=[]
for index in id_list:
    tempdf=graph.run(f"\
    MATCH (n)\
    WHERE ID(n)={index[0]}\
    MATCH (u:User) WHERE {index[1]}<u.useful<={index[2]}\
    RETURN n.useful, u.name, u.useful LIMIT 6").to_data_frame()
    dflist.append(tempdf)

In [37]:
dflist;

In [38]:
#this is for adding the useful edges
# id_list=[(6674,0,1),(6680,1,3),(6681,3,7),(6682,7,19),(6683,110,999999)]
# dflist=[]



# for index in id_list:
#     tempdf=graph.run(f"\
#     MATCH (n)\
#     WHERE ID(n)={index[0]}\
#     MATCH (u:User) WHERE {index[1]}<u.useful<={index[2]}\
#     CREATE (u)-[:USEFUL_LEVEL]->(n)").to_data_frame()
#     dflist.append(tempdf)


maybe put ghost users back in graph?

In [ ]:
UNWIND split(friends,',') as friend
MERGE (u1:User{id:friend})
MERGE (u)-[:FRIEND]-(u1)